# Bakery - Orders
In this problem we have to design an algorithm that will help bakery find the appropriate number of prepackaged bunch of items such that they use the greatest bunch most in order to save packaging money.<br>
This bakery has 3 menu items and the following bunches available:<br>

| Item name | Item code | Bunches|
|-----------|-----------|----------|
|Vegemite Scroll| VS5|3 @ 6.99, 5 @ 8.99|
| Croissant | CF | 2 @ 9.95, 5@ 16.95, 8 @ 24.95|
    |Bluberry Muffin|MB11|3 @ 5.95, 5 @ 9.95 ,9 @ 16.99|







In [1]:
class Bakery:
    def __init__(self,menu_items, menu_price_dict,printOrder):
        '''Initialize Bakery class with menu items, their item code and package with prices
        Parameters
        ----------
        menu_items: dict
            This is expected to contain item code (str) as key and item name as value
        menu_price_dict: dict
            This is expected to contain item code (str) as key and dictionary of package (int):price(float) as value
        printOrder: list
            Contains item codes (str) in order in which it is expected to be printed
             
        
        '''
#         menu_items = {'Vegemite Scroll':'VS5','Blueberry Muffin':'MB11','Croissant':'CF'}
        assert len(menu_price_dict)==len(menu_items), 'Number of items in menu and menu price do not match.'
        self.__items=menu_items
        self.__items_price_dict = menu_price_dict
        self.__valid_package_dict = {k:sorted(list(v.keys())) for k,v in menu_price_dict.items()}
        self.__printOrder = printOrder
        

    def __determineItemBreakup(self,valid_package_sizes,order_amount,min_package_req_for_diff_amts,pkgs_used):
        ''' Finds the minimum number of biggest packages required for provided order amounts to save space and
        returns the packages found
        Parameters
        ----------
        valid_package_sizes: list
            ascending list of valid package size
        order_amount: int
            number of items ordered for a particular menu item
        min_package_req_for_diff_amts: list
            initialized list of [0] with number of elements = order_amount +1 . Used to store minimum no. of packages 
            required for different order amts
        pkgs_used: list
            initialized list of [0] with number of elements = order_amount +1 . Used to track the type of packages for different
            order amounts which is used to calculate the different types of package used for an order amount
        
        Returns:
        -------
        pkgs_used: list
            list which now contains the package types for different order amounts which is used to 
            calculate the different types of package used for an order amount by __getPackagesUsedAndTotalForAmt method

        '''

        # we will try to find minimum package count for the each of different number of order amounts upto the required ordered amount
        # eg. if ordered amt = 10 then we will find min pkgs req for all amt till 10 i.e. 0,1,2,...10
        # We do this by taking each of these amts (0,1,2..10) and then we try to find the minimum no. of pkgs required by trying out 
        # each of the given valid sizes. 
        for temp_amt in range(min(valid_package_sizes),order_amount+1):
            #Initially assume maximum (approx) number of smallest packages
            pcks_used_count = temp_amt//min(valid_package_sizes)
            new_pkg =0
            validAmt = False
            # Now we will try to work our way upto the required package amount  and find minimum package numbers required
            # and store that for final output

            for valid_pk_sz in [valid_pk_size for valid_pk_size in valid_package_sizes if valid_pk_size<= temp_amt]:

                # So we assume we have used 1 of the valid pkg (which is why we deducting valid pkg in list)
                # And then we find minimum pkgs stored in list for amt - a valid pk size
                # If minimum package is the initialized value then we 
                # and store it in a list

                # These conditions ensure that the amount order can be served using the available valid package size list
                # Last condition ensure that minimum number of packages are chosen 

                if( ( ((temp_amt - valid_pk_sz >= min (valid_package_sizes))  & (min_package_req_for_diff_amts[temp_amt - valid_pk_sz] >0)) | 
                     (temp_amt - valid_pk_sz ==0) ) & 
                   (min_package_req_for_diff_amts[temp_amt - valid_pk_sz] + 1 <= pcks_used_count)):

                    pcks_used_count = min_package_req_for_diff_amts[temp_amt - valid_pk_sz] + 1
                    # recording the valid package size used
                    new_pkg = valid_pk_sz
                    # validAmt ensures that invalid order amounts are given 0 value for min_package_req_for_diff_amts
                    validAmt =True
            # updating valid packs used list and minimum number of packages used list
            min_package_req_for_diff_amts[temp_amt]=pcks_used_count if validAmt else 0
            # Adding the last package type to pkgs_used list
            pkgs_used[temp_amt] = new_pkg
        return pkgs_used
                
    

    def printBakeryReceipt(self,orders_dict):
        '''Prints Bakery receipt in provided format
        orders_dict: dict
            contains key = item code (str) value = order amounts(int)
        '''
        order_dict = {}
        assert len(orders_dict) <= len(self.__items) ,'Please check your order'
        
        # Creating order dictionary
        for k,v in orders_dict.items():
            order_dict[str.upper(k)] = int(v)
        # Creating order breakdown dictionary 
        order_breakdown_dict = {k:[] for k,v in self.__items.items()}
        order_total_dict = {k:0 for k,v in self.__items.items()}
        # Determining item breakup for each item ordered
        for k,v in order_dict.items():
            pkgUsedList=self.__determineItemBreakup(self.__valid_package_dict[k],order_dict[k],[0]*(order_dict[k]+1),[0]*(order_dict[k]+1))
            order_breakdown_dict[k],order_total_dict[k] = self.__getPackagesUsedAndTotalForAmt(k,pkgUsedList,order_dict[k])
            
        # Printing the results
        for k in self.__printOrder:
            print ('{} {} ${:g}'.format(order_dict[k],k,order_total_dict[k]))
            
            for i in reversed(sorted(list(set(order_breakdown_dict[k])))):
                if(order_total_dict[k]==0):
                    print ('{:>17}'.format('Invalid Order'))
                    continue
                print ('{:>7} x {} ${:g}'.format(order_breakdown_dict[k].count(i),i,self.__items_price_dict[k][i]))
        

        
        
    def __getPackagesUsedAndTotalForAmt(self,k,pkgUsed,amt):
        '''Calculates the package types to be used for order amount as well as the total bill
        Parameters:
        -----------
        k: str
            item code
        pkgUsed: list
            list which now contains the package types for different order amounts
        amt: int
            order amount for k item code
        Returns:
        --------
        pgkUsedList: list
            package types that will be provided to customer 
        
        total: float
            total bill calculated
        '''        
        pgkUsedList = []
        pkg = amt
        total =0
        
        while pkg > 0:
            if(pkgUsed[pkg])==0:
                pgkUsedList.append(0)
                break
            thisPkg = pkgUsed[pkg]
            pgkUsedList.append(thisPkg)
            total = total + self.__items_price_dict[k][thisPkg]
#             print(thisPkg)
            pkg = pkg - thisPkg
        pgkUsedList.sort(reverse=True)
        return pgkUsedList,total


In [2]:
# Initialising bakery with prices and menu
bakery = Bakery(menu_items = {'VS5':'Vegemite Scroll','MB11':'Blueberry Muffin','CF':'Croissant'},
               menu_price_dict = {'VS5':{3:6.99,5:8.99},'MB11':{2:9.95,5:16.95,8:24.95},'CF':{3:5.95,5:9.95,9:16.99}},
               printOrder = ['VS5','MB11','CF'])


#### Test case 1 (Given test case)
Input:<br>
10 VS5<br>
14 MB11<br>
13 CF<br>

Since jupyter gets stuck if there is a input() cell and I run all, so I have provided input already in a list

In [3]:
order_list = ['10 VS5',
             '14 MB11',
             '13 CF']
#for i in range(3):
#    order_list.append(input()) 

In [4]:
# Creating order dictionary 
order_dict = {x.split()[1]:int(x.split()[0]) for x in order_list }

In [5]:
order_dict

{'CF': 13, 'MB11': 14, 'VS5': 10}

In [6]:
bakery.printBakeryReceipt(order_dict)

10 VS5 $17.98
      2 x 5 $8.99
14 MB11 $54.8
      1 x 8 $24.95
      3 x 2 $9.95
13 CF $25.85
      2 x 5 $9.95
      1 x 3 $5.95


#### Test case 2 - order for a group of friends
Input:<br>
22 VS5<br>
30 MB11<br>
19 CF<br>

In [7]:
order_list2 = ['22 VS5',
             '30 MB11',
             '19 CF']
# for i in range(3):
#     order_list2.append(input()) 

In [8]:
# Creating order dictionary 
order_dict2 = {x.split()[1]:int(x.split()[0]) for x in order_list2 }

In [9]:
order_dict2

{'CF': 19, 'MB11': 30, 'VS5': 22}

In [10]:
bakery.printBakeryReceipt(order_dict2)

22 VS5 $45.94
      2 x 5 $8.99
      4 x 3 $6.99
30 MB11 $104.7
      3 x 8 $24.95
      3 x 2 $9.95
19 CF $36.89
      1 x 9 $16.99
      2 x 5 $9.95


#### Test case 3 - invalid order  (which cannot be purchased using prepackaged bunches)
Input:<br>
7 VS5<br>
1 MB11<br>
3 CF<br>

In [11]:
order_list3 = ['7 VS5',
             '1 MB11',
             '3 CF']
# for i in range(3):
#     order_list3.append(input()) 

In [12]:
# Creating order dictionary 
order_dict3 = {x.split()[1]:int(x.split()[0]) for x in order_list3 }

In [13]:
bakery.printBakeryReceipt(order_dict3)

7 VS5 $0
    Invalid Order
1 MB11 $0
    Invalid Order
3 CF $5.95
      1 x 3 $5.95


#### Test case 4 - Random large order for whole victoria
Input:<br>
534 VS5<br>
677 MB11<br>
1111 CF<br>

In [14]:
order_list4 = ['534 VS5',
             '677 MB11',
             '1111 CF']
# for i in range(3):
#     order_list4.append(input()) 

In [15]:
# Creating order dictionary 
order_dict4 = {x.split()[1]:int(x.split()[0]) for x in order_list4 }

In [16]:
bakery.printBakeryReceipt(order_dict4)

534 VS5 $964.92
    105 x 5 $8.99
      3 x 3 $6.99
677 MB11 $2112.75
     84 x 8 $24.95
      1 x 5 $16.95
1111 CF $2098.63
    122 x 9 $16.99
      2 x 5 $9.95
      1 x 3 $5.95
